In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

# Athletes

In [ ]:
athlete_dataset = pd.read_json('../olympic_athletes.json')
athlete_dataset.head()
athlete_dataset.shape
athlete_dataset.isnull().sum()

# Hosts

In [ ]:
hosts_dataset = pd.read_xml('../olympic_hosts.xml')
hosts_dataset.head()
hosts_dataset.shape
hosts_dataset.columns
hosts_dataset.drop(['index'], axis=1, inplace=True)
hosts_dataset.columns
hosts_dataset.isnull().sum()

# Medals

In [ ]:
medals_dataset = pd.read_excel('../olympic_medals.xlsx')
medals_dataset.shape
medals_dataset.isnull().sum()
medals_dataset.duplicated().sum()
medals_dataset.columns
medals_dataset = medals_dataset.rename({'slug_game': 'game_slug'}, axis=1)
medals_dataset.columns
medals_dataset.loc[(medals_dataset.duplicated())]
medals_dataset.loc[(medals_dataset.game_slug == 'paris-1900') & (medals_dataset.discipline_title == 'Polo')]
medals_dataset.loc[(medals_dataset.game_slug == 'london-1908') & (medals_dataset.discipline_title == 'Polo')]
medals_dataset.loc[(medals_dataset.game_slug == 'london-1908') & (medals_dataset.discipline_title == 'Hockey') & (
        medals_dataset.event_title == 'hockey men')]
medals_dataset.participant_title.unique()
medals_dataset.drop(['Unnamed: 0', 'participant_title', 'athlete_url'], axis=1, inplace=True)
medals_dataset.tail()

In [ ]:
results_dataset = pd.read_html('../olympic_results.html')[0]
results_dataset.head()
results_dataset.shape
results_dataset.isnull().sum()
results_dataset.duplicated().sum()
results_dataset.loc[(results_dataset.duplicated())]
results_dataset.value_unit.unique()
results_dataset.value_type.unique()
results_dataset.drop(['Unnamed: 0', 'athlete_url'], axis=1, inplace=True)
results_dataset = results_dataset.rename({'slug_game': 'game_slug'}, axis=1)
results_dataset.columns

# Separate winter and summer games

In [ ]:
game_types = hosts_dataset[['game_slug', 'game_season', 'game_year']]
merged_hosts_results = results_dataset.merge(game_types, on='game_slug')
merged_hosts_results.head()
merged_hosts_results.shape
merged_hosts_results.game_season.unique()

summer_results = merged_hosts_results.loc[(merged_hosts_results.game_season == 'Summer')].copy()
winter_results = merged_hosts_results.loc[(merged_hosts_results.game_season == 'Winter')].copy()
summer_results.drop(['game_season'], axis=1, inplace=True)

summer_results.head()
summer_results.shape
summer_results.isnull().sum()
summer_results.duplicated().sum()
duplicate_summer_results = summer_results.loc[(summer_results.duplicated())]
duplicate_summer_results

# Calculate the total number of each type of medals for each games by country

In [ ]:
summer_results.medal_type.unique()
summer_results['medal_type'] = summer_results['medal_type'].fillna('None')
summer_results['total_medals'] = summer_results['medal_type'].apply(lambda x: 0 if x == 'None' else 1)
summer_results['gold_medals'] = summer_results['medal_type'].apply(lambda x: 1 if x == 'GOLD' else 0)
summer_results['silver_medals'] = summer_results['medal_type'].apply(lambda x: 1 if x == 'SILVER' else 0)
summer_results['bronze_medals'] = summer_results['medal_type'].apply(lambda x: 1 if x == 'BRONZE' else 0)
summer_results
summer_results.country_name.unique()
medals_by_country = summer_results.groupby(['game_year', 'country_name']).agg(
    {'total_medals': 'sum', 'gold_medals': 'sum', 'silver_medals': 'sum', 'bronze_medals': 'sum'})
medals_by_country = medals_by_country.sort_values(by=['game_year', 'total_medals'],
                                                  ascending=[True, False]).reset_index()
medals_by_country

# Calculate the total number of disciplines for each games by country

In [ ]:
sports_by_country = summer_results.groupby(['game_year', 'country_name', 'discipline_title']).count()
sports_by_country
sports_by_country = sports_by_country[['event_title']].reset_index()
sports_by_country = sports_by_country.rename({'discipline_title': 'sports', 'event_title': 'events'},
                                             axis=1)
sports_by_country
sports_by_country = sports_by_country.groupby(['game_year', 'country_name']).agg({'sports': 'count'})
sports_by_country = sports_by_country.reset_index()
sports_by_country
events_by_country = summer_results.groupby(['game_year', 'country_name', 'event_title']).count()
events_by_country

events_by_country = events_by_country[['discipline_title']].reset_index()
events_by_country = events_by_country.rename(
    {'event_title': 'events', 'discipline_title': 'participation'}, axis=1)
events_by_country = events_by_country.groupby(['game_year', 'country_name']).agg(
    {'events': 'count'})
events_by_country = events_by_country.reset_index()
events_by_country
olympic_data = medals_by_country.merge(sports_by_country, on=['game_year', 'country_name'])
olympic_data = olympic_data.merge(events_by_country, on=['game_year', 'country_name'])
olympic_data = olympic_data.sort_values(by=['game_year', 'total_medals'], ascending=[True, False])
olympic_data

In [41]:
games_participation = summer_results.groupby(['country_name', 'game_year']).agg(
    {'total_medals': 'sum', 'gold_medals': 'sum', 'silver_medals': 'sum', 'bronze_medals': 'sum'})
games_participation
games_participation_reset = games_participation.reset_index()
games_participation_france = games_participation_reset.loc[(games_participation_reset.country_name == 'France')]
games_participation_france
import warnings

warnings.filterwarnings('ignore')
games_participation_france = games_participation_reset.loc[(games_participation_reset.country_name == 'France')]
games_participation_france['game_part'] = range(0, games_participation_france.shape[0])
games_participation_france['previous_game_medal'] = games_participation_france['total_medals'].shift(1, fill_value=0)
games_participation_france['previous_game_gold'] = games_participation_france['gold_medals'].shift(1, fill_value=0)
games_participation_france['previous_game_silver'] = games_participation_france['silver_medals'].shift(1, fill_value=0)
games_participation_france['previous_game_bronze'] = games_participation_france['bronze_medals'].shift(1, fill_value=0)

games_participation_france
games_participation_reset
country_list = list(games_participation_reset.country_name.unique())
country_list.remove('France')
country_list
for country in country_list:
    temp_games_participation = games_participation_reset.loc[(games_participation_reset.country_name == country)]
    temp_games_participation['game_part'] = range(0, temp_games_participation.shape[0])
    temp_games_participation['previous_game_medal'] = temp_games_participation['total_medals'].shift(1, fill_value=0)
    temp_games_participation['previous_game_gold'] = temp_games_participation['gold_medals'].shift(1, fill_value=0)
    temp_games_participation['previous_game_silver'] = temp_games_participation['silver_medals'].shift(1, fill_value=0)
    temp_games_participation['previous_game_bronze'] = temp_games_participation['bronze_medals'].shift(1, fill_value=0)
    games_participation_france = pd.concat([games_participation_france, temp_games_participation])
games_participation_france.columns
games_participation_france = games_participation_france[
    ['game_year', 'country_name', 'total_medals', 'gold_medals', 'silver_medals', 'bronze_medals', 'game_part',
     'previous_game_medal', 'previous_game_gold', 'previous_game_silver', 'previous_game_bronze']]
games_participation_france = games_participation_france.sort_values(by=['game_year', 'total_medals'], ascending=[True, False])
games_participation_france
games_participation_france = games_participation_france[
    ['game_year', 'country_name', 'game_part', 'previous_game_medal', 'previous_game_gold', 'previous_game_silver',
     'previous_game_bronze']]
games_participation_france
olympic_data = pd.merge(olympic_data, games_participation_france, on=['game_year', 'country_name'])

olympic_data = olympic_data.sort_values(by=['game_year', 'total_medals'], ascending=[True, False])

# Training set to prepare for regression models

In [42]:
country_names = list(olympic_data.country_name.unique())
country_dict = {}
for idx, country in enumerate(country_names):
    country_dict[country] = idx
country_dict

train_data = olympic_data.loc[(olympic_data.game_year < 2020)]
test_data = olympic_data.loc[(olympic_data.game_year == 2020)]

X_train = train_data[
    ['country_name', 'sports', 'events', 'game_part', 'previous_game_medal', 'previous_game_gold', 'previous_game_silver',
     'previous_game_bronze']]
X_test = test_data[
    ['country_name', 'sports', 'events', 'game_part', 'previous_game_medal', 'previous_game_gold', 'previous_game_silver',
     'previous_game_bronze']]

y_train_total = train_data['total_medals']
y_train_gold = train_data['gold_medals']
y_train_silver = train_data['silver_medals']
y_train_bronze = train_data['bronze_medals']

y_test_total = test_data['total_medals']
y_test_gold = test_data['gold_medals']
y_test_silver = test_data['silver_medals']
y_test_bronze = test_data['bronze_medals']

X_train = X_train.replace(country_dict)
X_test = X_test.replace(country_dict)

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

linear_model = LinearRegression()
linear_model.fit(X_train, y_train_total)
linear_predictions = linear_model.predict(X_test)

print('Predicted labels : ', np.round(linear_predictions)[:10])
print('Actual labels : ', y_test_total[:10])

linear_mse = mean_squared_error(y_test_total, linear_predictions)
print("MSE:", linear_mse)

linear_rmse = np.sqrt(linear_mse)
print("RMSE:", linear_rmse)

linear_r2 = r2_score(y_test_total, linear_predictions)
print("R2:", linear_r2)

results_2020_total_medals = pd.DataFrame(
    {'country': test_data['country_name'], 'linear_total_medals_pred': np.round(linear_predictions),
     'total_medals_actual': test_data['total_medals']})

pd.set_option('display.max_rows', None)

results_2020_total_medals

### Results

In [ ]:
results_2020_total_medals = results_2020_total_medals[['country', 'linear_total_medals_pred', 'total_medals_actual']]
results_2020_total_medals